# Машинное обучение в потоке данных. Регрессия

Реализация градиентного спуска, способного выполнять обработку и оценку качества в потоке данных. Версия для обработки большого массива данных на локальной машине.

In [8]:
import csv, time, os
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction import FeatureHasher

source = "\\bikesharing\\hour.csv"
local_path = os.getcwd()
SEP = ","

def apply_log(x): return np.log(float(x)+1)
def apply_exp(x): return np.exp(float(x))-1

SGD = SGDRegressor(loss="squared_loss", penalty=None, random_state=1, average=True)
h = FeatureHasher(non_negative=True)
val_rmse = 0
val_rmsle = 0
predictions_start = 16000

with open(local_path+"\\"+source, "r") as R:           
    iterator = csv.DictReader(R, delimiter=SEP)
    
    for n, row in enumerate(iterator):
        # ОБРАБОТКА ДАННЫХ
        target = np.array([apply_log(row["cnt"])])
        
        features = {k+"_"+v:1 
                    for k,v in row.items() # в Python2 ->  row.iteritems()
                    if k in ['holiday','hr','mnth','season', 'weathersit','weekday','workingday','yr']}
        
        numeric_features = {k:float(v)    # в Python2 ->  row.iteritems()
                            for k,v in row.items() if k in ['hum', 'temp', 'atemp', 'windspeed']}
        
        features.update(numeric_features)
        hashed_features = h.transform([features])
        
        # МАШИННОЕ ОБУЧЕНИЕ
        if (n+1) >= predictions_start:
            
            # ФАЗА С ОТКЛАДЫВАНИЕМ ПОСЛЕ N
            predicted = SGD.predict(hashed_features)
            val_rmse += (apply_exp(predicted) - apply_exp(target))**2
            val_rmsle += (predicted - target)**2
            
            if (n-predictions_start+1) % 250 == 0 and (n+1) > predictions_start:
                print("%s RMSE на отложенных данных: %0.3f" 
                       % (time.strftime('%X'), (val_rmse / float(n-predictions_start+1))**0.5)),
                print("RMSLE на отложенных данных: %0.3f" 
                       % ((val_rmsle / float(n-predictions_start+1))**0.5))
        else:
            # ФАЗА ОБУЧЕНИЯ
            SGD.partial_fit(hashed_features, target)
            
print("%s ИТОГОВАЯ RMSE на отложенных данных: %0.3f" 
       % (time.strftime("%X"), (val_rmse / float(n-predictions_start+1))**0.5))
print("%s ИТОГОВАЯ RMSLE на отложенных данных: %0.3f" 
       % (time.strftime("%X"), (val_rmsle / float(n-predictions_start+1))**0.5))

22:15:44 RMSE на отложенных данных: 281.065
RMSLE на отложенных данных: 1.899
22:15:44 RMSE на отложенных данных: 254.958
RMSLE на отложенных данных: 1.800
22:15:44 RMSE на отложенных данных: 255.456
RMSLE на отложенных данных: 1.798
22:15:44 RMSE на отложенных данных: 254.563
RMSLE на отложенных данных: 1.818
22:15:45 RMSE на отложенных данных: 239.740
RMSLE на отложенных данных: 1.737
22:15:45 ИТОГОВАЯ RMSE на отложенных данных: 229.274
22:15:45 ИТОГОВАЯ RMSLE на отложенных данных: 1.678
